<a href="https://colab.research.google.com/github/fboldt/aulasml/blob/master/aula7a_arvore_atributos_discretos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install ucimlrepo

In [23]:
from ucimlrepo import fetch_ucirepo

car_evaluation = fetch_ucirepo(id=19)

X = car_evaluation.data.features.to_numpy()
y = car_evaluation.data.targets.to_numpy()[:,0]

print(car_evaluation.variables)

       name     role         type demographic  \
0    buying  Feature  Categorical        None   
1     maint  Feature  Categorical        None   
2     doors  Feature  Categorical        None   
3   persons  Feature  Categorical        None   
4  lug_boot  Feature  Categorical        None   
5    safety  Feature  Categorical        None   
6     class   Target  Categorical        None   

                                         description units missing_values  
0                                       buying price  None             no  
1                           price of the maintenance  None             no  
2                                    number of doors  None             no  
3              capacity in terms of persons to carry  None             no  
4                           the size of luggage boot  None             no  
5                        estimated safety of the car  None             no  
6  evaulation level (unacceptable, acceptable, go...  None             no  

In [24]:
set(y), len(y)

({'acc', 'good', 'unacc', 'vgood'}, 1728)

In [25]:
combinacoes = 1
for i in range(X.shape[1]):
  valores = set(X[:,i])
  combinacoes *= len(valores)
  print(f"{i}: {valores}")
print(combinacoes)

0: {'low', 'vhigh', 'high', 'med'}
1: {'low', 'vhigh', 'high', 'med'}
2: {'2', '4', '3', '5more'}
3: {'2', '4', 'more'}
4: {'small', 'big', 'med'}
5: {'low', 'high', 'med'}
1728


In [26]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from collections import Counter
import numpy as np

def maisFrequente(y):
  counter = Counter(y)
  maisFreq = counter.most_common(1)[0][0]
  return maisFreq

class ZeroR(BaseEstimator, ClassifierMixin):
  def fit(self, X, y):
    self.resposta = maisFrequente(y)
    return self

  def predict(self, X):
    y = np.empty(X.shape[0], dtype='<U5')
    y.fill(self.resposta)
    return y

modelo = ZeroR()
modelo.fit(X, y)
modelo.predict(X)
accuracy_score(y, modelo.predict(X))

0.7002314814814815

In [27]:
maisFrequente(y)

'unacc'

In [28]:
sum(y=='unacc')/len(y)

0.7002314814814815

In [29]:
# apenas uma característica
class Arvore(BaseEstimator, ClassifierMixin):
  def fit(self, X, y):
    self.caracteristica_do_no = 0
    self.valor_do_no = list(set(X[:,self.caracteristica_do_no]))[0]
    exemplos_com_valores_iguais_ao_no = X[:,self.caracteristica_do_no] == self.valor_do_no
    if sum(exemplos_com_valores_iguais_ao_no)>0 and sum(~exemplos_com_valores_iguais_ao_no)>0:
      self.arvore_de_exemplos_com_valores_iguais = Arvore()
      self.arvore_de_exemplos_com_valores_iguais.fit(X[exemplos_com_valores_iguais_ao_no],
                                                    y[exemplos_com_valores_iguais_ao_no])
      self.arvore_de_exemplos_com_valores_diferentes = Arvore()
      self.arvore_de_exemplos_com_valores_diferentes.fit(X[~exemplos_com_valores_iguais_ao_no],
                                                        y[~exemplos_com_valores_iguais_ao_no])
    else:
      self.resposta = maisFrequente(y)
    return self

  def predict(self, X):
    y = np.empty(X.shape[0], dtype='<U5')
    if hasattr(self, 'resposta'):
      y.fill(self.resposta)
    else:
      exemplos_com_valores_iguais_ao_no = X[:,self.caracteristica_do_no] == self.valor_do_no
      y[exemplos_com_valores_iguais_ao_no] = self.arvore_de_exemplos_com_valores_iguais.predict(X[exemplos_com_valores_iguais_ao_no])
      y[~exemplos_com_valores_iguais_ao_no] = self.arvore_de_exemplos_com_valores_diferentes.predict(X[~exemplos_com_valores_iguais_ao_no])
    return y

modelo = Arvore()
modelo.fit(X, y)
ypred = modelo.predict(X)
accuracy_score(y, ypred)

0.7002314814814815

In [30]:
def print_arvore(arvore):
  print(f"Característica: {arvore.caracteristica_do_no}, Valor: {arvore.valor_do_no}")
  if hasattr(arvore, 'resposta'):
    print(arvore.resposta)
  else:
    print_arvore(arvore.arvore_de_exemplos_com_valores_iguais)
    print_arvore(arvore.arvore_de_exemplos_com_valores_diferentes)

print_arvore(modelo)

Característica: 0, Valor: low
Característica: 0, Valor: low
unacc
Característica: 0, Valor: vhigh
Característica: 0, Valor: vhigh
unacc
Característica: 0, Valor: high
Característica: 0, Valor: high
unacc
Característica: 0, Valor: med
unacc


In [31]:
def impureza(y): #Gini
  labels = list(set(y))
  labels.sort()
  probabilidades = np.zeros((len(labels),))
  for i in range(len(labels)):
    probabilidades[i] = sum(y==labels[i])/len(y)
  result = 1 - sum(probabilidades**2)
  return result

ytmp = y[:]
valor = impureza(ytmp)
print(valor)

0.457283763074417


In [32]:
def impurezaValor(x, y, valor):
  iguais = x == valor
  impurezaIguais = impureza(y[iguais])
  impurezaDiferentes = impureza(y[~iguais])
  propIguais = sum(iguais)/len(y)
  propDiferentes = sum(~iguais)/len(y)
  result = propIguais*impurezaIguais + propDiferentes*impurezaDiferentes
  return result

print(impurezaValor(X[:,0], y, 'vhigh'))


0.44934645776177407


In [33]:
def impurezaMinima(X, y):
  impurezas = []
  caracteristicaValores = []
  for i in range(X.shape[1]):
    valores = sorted(list(set(X[:,i])))
    for valor in valores:
      caracteristicaValores.append([i, valor])
      impurezas.append(impurezaValor(X[:,i], y, valor))
  caracteristicaValores = np.array(caracteristicaValores)
  impurezas = np.array(impurezas)
  indiceMenorImpureza = np.argmin(impurezas)
  caracteristica, valor = caracteristicaValores[indiceMenorImpureza]
  return impurezas[indiceMenorImpureza], int(caracteristica), valor

impurezaMinima(X, y)

(0.38615712609310704, 3, '2')

In [34]:
class Arvore(BaseEstimator, ClassifierMixin):
  def fit(self, X, y):
    self.impureza, self.caracteristica_do_no, self.valor_do_no = impurezaMinima(X, y)
    exemplos_com_valores_iguais_ao_no = X[:,self.caracteristica_do_no] == self.valor_do_no
    if sum(exemplos_com_valores_iguais_ao_no)>0 and sum(~exemplos_com_valores_iguais_ao_no)>0:
      self.arvore_de_exemplos_com_valores_iguais = Arvore()
      self.arvore_de_exemplos_com_valores_iguais.fit(X[exemplos_com_valores_iguais_ao_no],
                                                    y[exemplos_com_valores_iguais_ao_no])
      self.arvore_de_exemplos_com_valores_diferentes = Arvore()
      self.arvore_de_exemplos_com_valores_diferentes.fit(X[~exemplos_com_valores_iguais_ao_no],
                                                        y[~exemplos_com_valores_iguais_ao_no])
    else:
      self.resposta = maisFrequente(y)
    return self

  def predict(self, X):
    y = np.empty(X.shape[0], dtype='<U5')
    if hasattr(self, 'resposta'):
      y.fill(self.resposta)
    else:
      exemplos_com_valores_iguais_ao_no = X[:,self.caracteristica_do_no] == self.valor_do_no
      y[exemplos_com_valores_iguais_ao_no] = self.arvore_de_exemplos_com_valores_iguais.predict(X[exemplos_com_valores_iguais_ao_no])
      y[~exemplos_com_valores_iguais_ao_no] = self.arvore_de_exemplos_com_valores_diferentes.predict(X[~exemplos_com_valores_iguais_ao_no])
    return y

modelo = Arvore()
modelo.fit(X, y)
ypred = modelo.predict(X)
accuracy_score(y, ypred)

1.0

In [35]:
from sklearn.model_selection import cross_validate

scores = cross_validate(Arvore(), X, y)
scores['test_score'], np.mean(scores['test_score'])

(array([0.62716763, 0.73121387, 0.75144509, 0.75362319, 0.8057971 ]),
 0.7338493758900897)

In [36]:
class Arvore(BaseEstimator, ClassifierMixin):
  def __init__(self, max_depth=None):
    self.max_depth = max_depth

  def fit(self, X, y):
    self.impureza, self.caracteristica_do_no, self.valor_do_no = impurezaMinima(X, y)
    exemplos_com_valores_iguais_ao_no = X[:,self.caracteristica_do_no] == self.valor_do_no
    if sum(exemplos_com_valores_iguais_ao_no)>0 and sum(~exemplos_com_valores_iguais_ao_no)>0 and (self.max_depth == None or self.max_depth > 0):
      max_depth = None if self.max_depth == None else self.max_depth - 1
      self.arvore_de_exemplos_com_valores_iguais = Arvore(max_depth)
      self.arvore_de_exemplos_com_valores_iguais.fit(X[exemplos_com_valores_iguais_ao_no],
                                                    y[exemplos_com_valores_iguais_ao_no])
      self.arvore_de_exemplos_com_valores_diferentes = Arvore(max_depth)
      self.arvore_de_exemplos_com_valores_diferentes.fit(X[~exemplos_com_valores_iguais_ao_no],
                                                        y[~exemplos_com_valores_iguais_ao_no])
    else:
      self.resposta = maisFrequente(y)
    return self

  def predict(self, X):
    y = np.empty(X.shape[0], dtype='<U5')
    if hasattr(self, 'resposta'):
      y.fill(self.resposta)
    else:
      exemplos_com_valores_iguais_ao_no = X[:,self.caracteristica_do_no] == self.valor_do_no
      y[exemplos_com_valores_iguais_ao_no] = self.arvore_de_exemplos_com_valores_iguais.predict(X[exemplos_com_valores_iguais_ao_no])
      y[~exemplos_com_valores_iguais_ao_no] = self.arvore_de_exemplos_com_valores_diferentes.predict(X[~exemplos_com_valores_iguais_ao_no])
    return y

modelo = Arvore(2)
modelo.fit(X, y)
ypred = modelo.predict(X)
print(accuracy_score(y, ypred))
scores = cross_validate(modelo, X, y)
scores['test_score'], np.mean(scores['test_score'])

0.7777777777777778


(array([0.6300578 , 0.71965318, 0.80346821, 0.85797101, 0.87826087]),
 0.7778822149618833)

In [37]:
print_arvore(modelo)

Característica: 3, Valor: 2
Característica: 0, Valor: high
Característica: 0, Valor: high
unacc
Característica: 0, Valor: low
unacc
Característica: 5, Valor: low
Característica: 0, Valor: high
unacc
Característica: 0, Valor: vhigh
acc


In [38]:
from sklearn.model_selection import StratifiedShuffleSplit

scores = cross_validate(modelo, X, y, cv=StratifiedShuffleSplit(n_splits=5, random_state=42))
scores['test_score'], np.mean(scores['test_score'])

(array([0.79768786, 0.76878613, 0.78034682, 0.80346821, 0.76878613]),
 0.7838150289017342)

In [39]:
class Arvore(BaseEstimator, ClassifierMixin):
  def __init__(self, max_depth=None, min_samples_split=2):
    self.max_depth = max_depth
    self.min_samples_split = min_samples_split

  def fit(self, X, y):
    self.impureza, self.caracteristica_do_no, self.valor_do_no = impurezaMinima(X, y)
    exemplos_com_valores_iguais_ao_no = X[:,self.caracteristica_do_no] == self.valor_do_no
    if sum(exemplos_com_valores_iguais_ao_no)>self.min_samples_split and sum(~exemplos_com_valores_iguais_ao_no)>self.min_samples_split and (self.max_depth == None or self.max_depth > 0):
      max_depth = None if self.max_depth == None else self.max_depth - 1
      self.arvore_de_exemplos_com_valores_iguais = Arvore(max_depth, self.min_samples_split)
      self.arvore_de_exemplos_com_valores_iguais.fit(X[exemplos_com_valores_iguais_ao_no],
                                                    y[exemplos_com_valores_iguais_ao_no])
      self.arvore_de_exemplos_com_valores_diferentes = Arvore(max_depth, self.min_samples_split)
      self.arvore_de_exemplos_com_valores_diferentes.fit(X[~exemplos_com_valores_iguais_ao_no],
                                                        y[~exemplos_com_valores_iguais_ao_no])
    else:
      self.resposta = maisFrequente(y)
    return self

  def predict(self, X):
    y = np.empty(X.shape[0], dtype='<U5')
    if hasattr(self, 'resposta'):
      y.fill(self.resposta)
    else:
      exemplos_com_valores_iguais_ao_no = X[:,self.caracteristica_do_no] == self.valor_do_no
      y[exemplos_com_valores_iguais_ao_no] = self.arvore_de_exemplos_com_valores_iguais.predict(X[exemplos_com_valores_iguais_ao_no])
      y[~exemplos_com_valores_iguais_ao_no] = self.arvore_de_exemplos_com_valores_diferentes.predict(X[~exemplos_com_valores_iguais_ao_no])
    return y

modelo = Arvore(None, 10)
modelo.fit(X, y)
ypred = modelo.predict(X)
print(accuracy_score(y, ypred))
scores = cross_validate(modelo, X, y)
scores['test_score'], np.mean(scores['test_score'])

0.9502314814814815


(array([0.6416185 , 0.72543353, 0.64739884, 0.79710145, 0.76521739]),
 0.7153539415263467)